<a href="https://colab.research.google.com/github/eduviana1/TEEP_Chuva_Horaria/blob/main/6_Krigagem_AlertaRio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install cartopy
!pip install pykrige

In [ ]:
import numpy as np
import xarray as xr
import glob
import os
import matplotlib.pyplot as plt
import cartopy.feature as cfeature
import cartopy.crs as ccrs
import pandas as pd
import geopandas as gpd
from google.colab import drive
drive.mount('/content/drive')
from pykrige.ok import OrdinaryKriging
from sklearn.linear_model import LinearRegression
import seaborn as sns
from tqdm import tqdm # Barra de Loading...

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Krigagem

In [ ]:
alertario = pd.read_csv('/content/drive/MyDrive/Grupo 4 - TEEP/Trabalho Prático - Parte II/UTC_chuva_horaria_alertario.csv')
coordenadas_estacoes = pd.read_csv("/content/drive/MyDrive/ic_fabricio/coordenadas_alertario.csv")

In [ ]:
alertario.head()

,DataHora,alto_da_boa_vista,anchieta,av_brasil_mendanha,bangu,barrinha,campo_grande,cidade_de_deus,copacabana,grajau,...,santa_cruz,santa_teresa,sao_cristovao,saude,sepetiba,tanque,tijuca,tijuca_muda,urca,vidigal
0,1997-01-01 04:00:00,NaN,0.0,NaN,0.0,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0
1,1997-01-01 05:00:00,NaN,0.0,NaN,0.0,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0
2,1997-01-01 06:00:00,NaN,0.0,NaN,0.0,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0
3,1997-01-01 07:00:00,NaN,0.0,NaN,0.0,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0
4,1997-01-01 08:00:00,NaN,0.0,NaN,0.0,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0


In [ ]:
coordenadas_estacoes.head()

,estacao,latitude,longitude,cota_m
0,vidigal,-22.99250,-43.23306,85
1,urca,-22.95583,-43.16667,90
2,rocinha,-22.98583,-43.24500,160
3,tijuca,-22.93194,-43.22167,340
4,santa_teresa,-22.93167,-43.19639,170


In [ ]:
# Verificando se todas as mesmas estações estão em ambos os arquivos
estacoes = alertario.drop('DataHora', axis = 1).columns.values
print(coordenadas_estacoes['estacao'].isin(estacoes).sum() == 33) #True

True


In [ ]:
dados_faltantes = alertario[alertario.isna().any(axis=1)]
dados_faltantes = dados_faltantes.drop('DataHora', axis = 1)
display(dados_faltantes.head())
print(dados_faltantes.shape)

,alto_da_boa_vista,anchieta,av_brasil_mendanha,bangu,barrinha,campo_grande,cidade_de_deus,copacabana,grajau,grajau_jacarepagua,...,santa_cruz,santa_teresa,sao_cristovao,saude,sepetiba,tanque,tijuca,tijuca_muda,urca,vidigal
0,NaN,0.0,NaN,0.0,NaN,0.0,0.0,0.0,0.0,NaN,...,0.0,0.0,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0
1,NaN,0.0,NaN,0.0,NaN,0.0,0.0,0.0,0.0,NaN,...,0.0,0.0,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0
2,NaN,0.0,NaN,0.0,NaN,0.0,0.0,0.0,0.0,NaN,...,0.0,0.0,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0
3,NaN,0.0,NaN,0.0,NaN,0.0,0.0,0.0,0.0,NaN,...,0.0,0.0,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0
4,NaN,0.0,NaN,0.0,NaN,0.0,0.0,0.0,0.0,NaN,...,0.0,0.0,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0


(141152, 33)


In [ ]:
def krig_drift(est_presentes, cota, prec):
  # Regressão Linear
  lr.fit(cota.reshape(-1,1), prec)
  # Obtendo reta de prec por cota
  a, b = lr.coef_, lr.intercept_
  # Calculcando residuos
  residuos = prec - lr.predict(cota.reshape(-1,1))

  x = coordenadas_estacoes[coordenadas_estacoes['estacao'].isin(est_presentes)]['longitude'].values
  y = coordenadas_estacoes[coordenadas_estacoes['estacao'].isin(est_presentes)]['latitude'].values

  OK = OrdinaryKriging(x = x,
  y = y,
  z = residuos,
  variogram_model= 'spherical' # Melhor representa fenômenos físicos
  )
  return a, b, OK

In [ ]:
estacoes = alertario.drop('DataHora', axis = 1).columns.values
lr = LinearRegression()
for indice, valores in tqdm(dados_faltantes.iterrows(), desc = 'Processando...'):

  est_presentes = valores[(valores.notna())].index.values
  est_faltantes = valores[valores.isna()].index.values
  # Krigagem Drift Externo
  # Estimando primeiro a tendencia (drift) que a cota (altura das estações) causa na precipitação
  cota = coordenadas_estacoes[coordenadas_estacoes['estacao'].isin(est_presentes)]['cota_m'].values
  prec = valores[est_presentes].values
  if prec.sum() == 0.0:
    dados_faltantes.loc[indice, est_faltantes] = 0.0
    continue
  else:
    # alpha e beta são os coeficientes da reta de previsão de precipitação a partir das cotas
    alpha, beta, OK = krig_drift(est_presentes,cota, prec)
    # Iterando entre estações faltantes
    for estacao in est_faltantes:
      # Coordenadas da estação com dados faltantes
      x_est, y_est = coordenadas_estacoes[coordenadas_estacoes['estacao'] == estacao][['longitude', 'latitude']].values[0]
      # Cota (altura) da estação com dados faltantes
      cota_est = coordenadas_estacoes[coordenadas_estacoes['estacao'] == estacao]['cota_m'].values[0]
      # Calculando com krigagem ordinaria os residuos (erros) esperados do ponto
      residuo_previsto, _ = OK.execute(style ='points',xpoints = [x_est], ypoints = [y_est])
      # Atribuindo valor previsto ao DataFrame na respectiva data (Tendencia + Resíduo)
      dados_faltantes.at[indice, estacao] = residuo_previsto + alpha*cota_est + beta

Processando...: 141152it [26:17, 89.46it/s] 


In [ ]:
# Verificando se tem valores negativos (devido a regressão linear da altitude)
dados_faltantes.describe()

In [ ]:
# Zerando valores negativos
dados_faltantes[dados_faltantes.select_dtypes(include='number').columns] = dados_faltantes[dados_faltantes.select_dtypes(include='number').columns].clip(lower = 0)

In [ ]:
alertario.update(dados_faltantes)

In [ ]:
alertario.to_csv('/content/drive/MyDrive/Grupo 4 - TEEP/Trabalho Prático - Parte II/UTC_chuva_horaria_alertario_krig.csv', index = False)